In [9]:
from math import log
from line_profiler import LineProfiler

In [10]:
%load_ext line_profiler

This sequence is used for division by multiplication with an approximation of the inverse of the divisor on computers if they support multiplying two 32-bit numbers for a 64-bit result.  
This is usually much faster than a division instruction because integer division is a very slow operation on most computers.

Source: https://en.m.wikipedia.org/wiki/Division_algorithm#Division_by_a_constant  
https://oeis.org/A346495 https://oeis.org/A346496  
https://stackoverflow.com/questions/5558492/divide-by-10-using-bit-shifts

In [1]:
def power_of_two(n):
    return n == (1<<(n.bit_length()-1))

In [2]:
def a(n):
    Result = 1
    if not power_of_two(n):
        n_c = ((1<<32) - ((1<<32) % n)) - 1
        b   = 1
        while (1<<b) <= (n_c * ((n - 1) - (((1<<b) - 1) % n))):
            b += 1
        Result = ((1<<b) + (n - 1) - (((1<<b) - 1) % n)) // n
    return Result

In [3]:
print([a(n) for n in range(1, 26)])

[1, 1, 2863311531, 1, 3435973837, 2863311531, 4908534053, 1, 954437177, 3435973837, 3123612579, 2863311531, 1321528399, 4908534053, 2290649225, 1, 4042322161, 954437177, 7233629131, 3435973837, 6544712071, 3123612579, 2987803337, 2863311531, 1374389535]


In [4]:
def b(n):
    Result = None
    if power_of_two(n):
        Result = n.bit_length() - 1
    else:
        n_c = ((1<<32) - ((1<<32) % n)) - 1
        b   = 1
        while (1<<b) <= (n_c * ((n - 1) - (((1<<b) - 1) % n))):
            b += 1
        Result = b
    return Result

In [5]:
print([b(n) for n in range(1, 67)])

[0, 1, 33, 2, 34, 34, 35, 3, 33, 35, 35, 35, 34, 36, 35, 4, 36, 34, 37, 36, 37, 36, 36, 36, 35, 35, 37, 37, 36, 36, 37, 5, 35, 37, 38, 35, 38, 38, 38, 37, 37, 38, 35, 37, 38, 37, 37, 37, 36, 36, 37, 36, 38, 38, 38, 38, 38, 37, 35, 37, 36, 38, 38, 6, 38, 36]


Smallest a(n) so that division by n can be performed by $$\left\lfloor\frac{x}{n}\right\rfloor = \frac{\lfloor(x*a(n)\rfloor}{2^{b(n)}}$$ for all $0 <= x < 2^{32}$. 

In [6]:
LIMIT     = 32
BIT_LIMIT = 1<<LIMIT
def seq(i):
    A = 1
    B = i.bit_length() - 1
    j = i - 1
    if not (i == (1<<B)):
        k = (BIT_LIMIT - (BIT_LIMIT % i)) - 1
        print(bin(k))
        B   = 1
        B2  = 1<<B
        while B2 <= (k * (j - ((B2 - 1) % i))):
            B += 1
            B2 = B2<<1
        A = (B2 + j - ((B2 - 1) % i)) // i
    return (A, B)

In [7]:
bin(1<<32)

'0b100000000000000000000000000000000'

In [8]:
print([seq(n) for n in range(1, 26)])

0b11111111111111111111111111111110
0b11111111111111111111111111111110
0b11111111111111111111111111111011
0b11111111111111111111111111111011
0b11111111111111111111111111111011
0b11111111111111111111111111111001
0b11111111111111111111111111111011
0b11111111111111111111111111111011
0b11111111111111111111111111110110
0b11111111111111111111111111111011
0b11111111111111111111111111111110
0b11111111111111111111111111111110
0b11111111111111111111111111111011
0b11111111111111111111111111111001
0b11111111111111111111111111101111
0b11111111111111111111111111111011
0b11111111111111111111111111111011
0b11111111111111111111111111110011
0b11111111111111111111111111101111
0b11111111111111111111111111101010
[(1, 0), (1, 1), (2863311531, 33), (1, 2), (3435973837, 34), (2863311531, 34), (4908534053, 35), (1, 3), (954437177, 33), (3435973837, 35), (3123612579, 35), (2863311531, 35), (1321528399, 34), (4908534053, 36), (2290649225, 35), (1, 4), (4042322161, 36), (954437177, 34), (7233629131, 37), (34359738

---

Finding the magic numbers in the way we did above is limited to 32 bits. But there is another way:
---

Source: Hacker's delight page 218 (chapter 10–15 Simple Code in Python)

Optimization: Switching from a linear search to a binary search reduced the runtime for inputs `m, p = magicgu(1<<1_000_000, 10**399)` from 40 minutes to 0.08 seconds, taking just 24 iterations to converge on the correct value...

In [11]:
def magicgu_slow(nmax, d):
    """Given the maximum value of the dividend nmax and the divisor d,
    returns a pair of integers:
        the magic number m and a shift amount p.
    To divide a dividend x by d, one multiplies x by m and then shifts the (full length) product right p bits.
    """
    dminus1      = d - 1
    nc           = (nmax // d) * dminus1
    search_range = 1 + (2 * nmax.bit_length())
    for p in range(0, search_range):
        q         = 1<<p
        qminus1   = q - 1
        remainder = (qminus1 % d)
        if q > (nc * (dminus1 - remainder)):
            m = ((q + dminus1) - remainder) // d
            return (m, p)
    raise Exception('Couldn\'t find a solution, something is wrong.')

In [12]:
def magicgu(nmax, d):
    """
    Given the maximum value of the dividend nmax and the divisor d, returns a pair of integers:
        the magic number m and a shift amount p.
    To divide a dividend x by d, one multiplies x by m and then shifts the (full length) product right p bits.
    This works because m / p will be very close to 1 / d, and p will be a power of two, making the divide a simple bit shift.
    """
    dminus1 = d - 1
    nc      = (nmax // d) * dminus1
    lowest  = 0
    highest = 1 + (2 * nmax.bit_length())
    while lowest < highest:
        p         = (highest + lowest) // 2
        q         = 1<<p
        qminus1   = q - 1
        remainder = (qminus1 % d)
        if q > (nc * (dminus1 - remainder)):
            highest = p
        else:
            lowest  = p + 1
    p         = (highest + lowest) // 2
    q         = 1<<p
    qminus1   = q - 1
    remainder = (qminus1 % d)
    m         = ((q + dminus1) - remainder) // d
    return (m, p)
        
    raise Exception('Couldn\'t find a solution, something is wrong.')

Example 1:

In [15]:
m, p = magicgu(10**399, 10); m, p

(292957247209924852137075224890370171032692958681326960826984919130001827539839214697757948051204538105646676347303848625162902618953473698102408269423433879425259310757283275210972425883538122452847336119254634643499505917091533837488298906776482947295647220679792987618467923311462718229513113755678107850012507556042175979861246759917806100226846569645613245309343003296469005736181778637822676173,
 1327)

In [16]:
x = 675498068352129731964884062763533329144415921218630285019127698639680940044270154133162810213979906631608629547659877222303938885553947675194806514177810321278967470122934933377995913130368225534489724572693654861728928532250033597205506631158056720564410536312378792296760105033917922985829102352323964365789020903947560236392997251191841134769795404868939332473278193403660671793915429281166670196

In [17]:
x < 10**399

True

In [18]:
d = 10

In [19]:
%%timeit
(x * m) >> p

2.24 µs ± 31.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [20]:
%%timeit
x // 10

164 ns ± 1.19 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


Example 2:

In [21]:
x = (1<<999_900)

In [22]:
d = 10**399

In [23]:
%%timeit
(x * m) >> p

1.38 ms ± 12.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [24]:
%%timeit
x // d

2.16 ms ± 15.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


This works, but sadly it's not really faster than the usual division, even when the number we're dividing is close to the maximum (which makes the difference between the mult and shift version and the division version the smallest).

---

If the divisor and the maximum size are fixed, we can go another approach which gets rid of even the multiply instruction and instead does a bunch of adds and shifts
---

Source: Hackers Delight (see above)

This again doesn't work for larger numbers. The given examples are only accurate for 32 bits (i think)

In [27]:
def divu10(n):
    q  = (n >> 1) + (n >> 2)
    q += (q >> 4)
    q += (q >> 8)
    q += (q >> 16)
    q  = (q >> 3)
    r  = n - (((q << 2) + q) << 1)
    return q + (r > 9), r

In [28]:
divu10(21)

(2, 11)

In [29]:
def divu100(n):
    q = (n >> 1) + (n >> 3) + (n >> 6) - (n >> 10) + (n >> 12) + (n >> 13) - (n >> 16)
    q = q + (q >> 20)
    q = q >> 6
    r = n - q*100
    return q + ((r + 28) >> 7)
    # // return q + (r > 99)

In [30]:
divu100(240)

2

In [31]:
def divu1000(n):
    t = (n >> 7) + (n >> 8) + (n >> 12)
    q = (n >> 1) + t + (n >> 15) + (t >> 11) + (t >> 14)
    q = q >> 9
    r = n - q*1000
    return q + ((r + 24) >> 10)
    # // return q + (r > 999)

In [32]:
divu1000(3600)

3